In [153]:
%autosave 0

# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import os
import pandas as pd
import pymongo
import time

Autosave disabled


In [195]:
# mongodb connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.job_description_db

In [155]:
# Initialize executable path for the chromedriver
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [156]:
base_url = 'https://stackoverflow.com/tags?page='

In [157]:
skill_list = []
ct_total_list = []
ct_week_list = []

In [158]:
for next_page in range(1,25):
    
    new_url = base_url + str(next_page) + "&tab=popular"
    browser.visit(new_url)
    html = browser.html
    soup = bs(html, 'html.parser')

    tag_card = soup.find('div', class_="grid-layout")
    
    for i in range(36):
        tag_name = tag_card.find_all('div', class_='grid-layout--cell')[i].a.text
        tag_question_ct = tag_card.find_all('span', class_='item-multiplier-count')[i].text
        
        try:
            tag_week_ct = tag_card.find_all('div', class_='grid--cell stats-row')[i]('a')[1].text
        except:
            tag_week_ct = "0 this week"     
        
        #print(tag_name + " " + tag_question_ct + " " + tag_week_ct)    
        
        if "-" in tag_name:
            tag_name = tag_name.replace("-", " ")    
        skill_list.append(tag_name)
        
        ct_total_list.append(int(tag_question_ct))
        
        if "week" in tag_week_ct:
            tag_week_ct = float(tag_week_ct.replace(' this week', ''))
        elif "month" in tag_week_ct:
            tag_week_ct = float(tag_week_ct.replace(' this month', ''))/4
        elif "year" in tag_week_ct:
            tag_week_ct = float(tag_week_ct.replace(' this year', ''))/52
        
        tag_week_ct = int(tag_week_ct)
        ct_week_list.append(tag_week_ct)

In [159]:
len(ct_week_list)


864

In [160]:
len(ct_total_list)

864

In [161]:
len(skill_list)

864

In [162]:
skill_df = pd.DataFrame({"Skill": skill_list, "Total Questions": ct_total_list, "Weekly Questions": ct_week_list})

In [168]:
skill_df.head(15)

,Skill,Total Questions,Weekly Questions
0,javascript,1793974,5142
1,java,1536932,3892
2,c#,1303086,2910
3,php,1276725,2689
4,android,1187331,2698
5,python,1150840,6175
6,jquery,950600,1145
7,html,816711,1952
8,c++,613555,1427
9,ios,595874,919


In [169]:
skill_wksort_df = skill_df.sort_values(by=['Weekly Questions'], ascending=False)

In [170]:
skill_weeksort_df = skill_wksort_df.reset_index(drop=True)

In [186]:
skill_week300_df = skill_weeksort_df.loc[skill_weeksort_df['Weekly Questions'] >= 300]

In [187]:
skill_week300_df

,Skill,Total Questions,Weekly Questions
0,python,1150840,6175
1,javascript,1793974,5142
2,java,1536932,3892
3,c#,1303086,2910
4,android,1187331,2698
5,php,1276725,2689
6,html,816711,1952
7,python 3.x,155033,1786
8,reactjs,133616,1457
9,c++,613555,1427


In [188]:
db.stackoverflow_skills.insert_many(skill_df.to_dict('records'))

In [198]:
for doc in db.jd:
    job_skills = []
    for tag in db.stackoverflow_skills.Skill:
        if tag in doc.job_description:
            job_skills.append(tag)
    db.jd.update({'job_skills': job_skills})

TypeError: 'Collection' object is not iterable